# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingenieria de Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 05**: Tiktok tracks analysis

**Fecha**:21/02/2025

**Nombre del Estudiante**:David Abraham Naranjo Salgado, Benjamin Leonardo Zarate Solano y Angel David Cortes Pacheco

**Profesor**: Pablo Camarillo Ramirez

## Description
In teams, write a Jupyter Notebook (spark cluster/notebooks/labs/lab05) to complete the following analysis tasks for the tiktok dataset using PySpark:

Filter and Count Popular Tracks. Filter songs with a popularity score greater than 80 and count the number of such tracks.

Calculate Average Duration of Songs by Genre. Group songs by genre and calculate the average duration mins for each genre.
Find the Top 5 Most Energetic Songs. Sort songs by energy in descending order and retrieve the top 5 songs.

Calculate the Total Duration of Songs in Each Playlist. Group songs by playlist name and calculate the total duration mins for
each playlist.

When you complete you Notebook, please submit a PR with the solution.

### Installing spark

In [1]:
import findspark
findspark.init()

### Create connection

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Tiktok-tracks-analysis") \
    .master("spark://b631371550f9:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

# Create SparkContext
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/22 01:16:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Import Spark_utils

In [3]:
from team_bellaco.spark_utils import SparkUtils

In [4]:
columns_info = [ ("track_id", "string"),
                ("track_name", "string"),
                ("artist_id", "string"),
                ("artist_name", "string"),
                ("album_id", "string"),
                ("duration", "integer"),
                ("release_date", "timestamp"),
                ("popularity", "integer"),
                ("danceability", "double"),
                ("energy", "double"),
                ("key", "integer"),
                ("loudness", "double"),
                ("mode", "integer"),
                ("speechiness", "double"),
                ("acousticness", "double"),
                ("instrumentalness", "double"),
                ("liveness", "double"),
                ("valence", "double"),
                ("tempo", "double"),
                ("playlist_id", "string"),
                ("playlist_name", "string"),
                ("duration_mins", "double"),
                ("genre", "string")]

schema = SparkUtils.generate_schema(columns_info)

# Create DataFrame
tiktok_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/tiktok.csv")

### Filter and Count Popular Tracks

In [5]:
filtered_popularity_df = tiktok_df.filter(tiktok_df["popularity"] > 80)
row_count = filtered_popularity_df.count()
row_count

1023

### Calculate Average Duration of Songs by Genre

In [6]:
from pyspark.sql.functions import col
grouped_df = tiktok_df.groupBy("genre").avg("duration")
grouped_df_with_minutes = grouped_df.withColumn("avg_duration_mins", col("avg(duration)") / 60)
grouped_df_with_minutes.show()

+------------------+------------------+------------------+
|             genre|     avg(duration)| avg_duration_mins|
+------------------+------------------+------------------+
|TIKTOK PHILIPPINES|196807.97061442563|3280.1328435737605|
|      TIKTOK DANCE|180901.24283501162|3015.0207139168606|
|           _TIKTOK|195071.78653012996|3251.1964421688326|
|        TIKTOK OPM|255431.57171314742|  4257.19286188579|
+------------------+------------------+------------------+



### Find the Top 5 Most Energetic Songs

In [8]:
from pyspark.sql.functions import desc
top5_df = tiktok_df.orderBy(col("energy").desc()).limit(5)
top5_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+----------+------------------+------------------+---+------------------+----+-----------+--------------------+-------------------+--------+------------------+------------------+--------------------+--------------------+-----------------+----------+
|            track_id|          track_name|           artist_id|         artist_name|            album_id|duration|       release_date|popularity|      danceability|            energy|key|          loudness|mode|speechiness|        acousticness|   instrumentalness|liveness|           valence|             tempo|         playlist_id|       playlist_name|    duration_mins|     genre|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+----------+------------------+------------------+---+------------------+----+-----------+--------

### Calculate the Total Duration of Songs in Each Playlist

In [ ]:
from pyspark.sql.functions import sum
total_duracion_df = tiktok_df.groupBy("playlist_name").agg(sum("duration"))
total_duracion_df.show()

+--------------------+-------------+
|       playlist_name|sum(duration)|
+--------------------+-------------+
|5IZc3KIVFhjzJ0L2k...|       448480|
|08ia51KbTcfs4QVT5...|       248910|
|7xVLFuuYdAvcTfcP3...|       567386|
|2RBILNmyq8p4fqVWO...|       129776|
|6GdDjthxbTGBV9rl2...|       199255|
|7krYEnB1OI1RbnJBa...|       125746|
|1FgPyHX7HruKDL4Tx...|       146688|
|62RtxFf9epYNWOUHJ...|       160166|
|5ow0sNF1zSqp71Ix5...|       164009|
|0LlJbV4lyzJYE14YC...|       642548|
|6NFKf8vBApSvtzkap...|       222446|
|5P8lyudWE7HQxb4lu...|       255040|
|2BgEsaKNfHUdlh97K...|       186986|
|7F9vK8hNFMml4GtHs...|       190427|
|4vVTI94F9uJ8lHNDW...|       201946|
|2uULRpRtKhCdojXwo...|       132000|
|1tRlGMHsf21FDo6pj...|       107403|
|215fAfwkWtlj30ofd...|       139285|
|3bidbhpOYeV4knp8A...|       258342|
|0YFocHKmrMme7Isel...|       288623|
+--------------------+-------------+
only showing top 20 rows



In [7]:
#sc.stop()